In [ ]:
%load_ext autoreload
%autoreload 2

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import ceil
from CIoTS import *
import time

In [13]:
max_ps = list(range(4, 9, 2))
test_ps = list(range(2, 11, 2))
runs = 20
dimensions = 4
data_length = 10000
alpha = 0.05

In [ ]:
f1_scores = pd.DataFrame(columns=['true_p', 'p', 'mean_f1', 'std_f1'])
runtimes = pd.DataFrame(columns=['true_p', 'p', 'mean_time', 'std_time'])

In [ ]:
for p in max_ps:
    incoming_edges = 3 #max(ceil(dimensions*p/3), 1)
    
    f1 = {p_test: [] for p_test in test_ps}
    runtime = {p_test: [] for p_test in test_ps}
    
    for run in range(runs):
        generator = CausalTSGenerator(dimensions=dimensions, max_p=p, data_length=data_length, incoming_edges=incoming_edges)
        ts = generator.generate()
            
        for p_test in test_ps:
            start_time = time.time()
            predicted_graph = pc_chen(partial_corr_test, ts, p_test, alpha)
            exec_time = time.time()-start_time
            
            f1[p_test].append(evaluate_edges(generator.graph, predicted_graph)['f1-score'])
            runtime[p_test].append(exec_time)
            print('done: p='+ str(p) + ' run='+str(run+1) + ' p_test='+str(p_test) + ' exec_time='+str(exec_time))
    
    for p_test in test_ps:
        f1_scores = f1_scores.append({'true_p': p, 'p': p_test, 'mean_f1': np.mean(f1[p_test]),
                                      'std_f1': np.std(f1[p_test])}, ignore_index=True)
        runtimes = runtimes.append({'true_p': p, 'p': p_test, 'mean_time': np.mean(runtime[p_test]),
                                    'std_time': np.std(runtime[p_test])}, ignore_index=True)

done: p=4 run=1 p_test=2 exec_time=3.8553874492645264
done: p=4 run=1 p_test=4 exec_time=21.73862886428833
done: p=4 run=1 p_test=6 exec_time=70.96734261512756
done: p=4 run=1 p_test=8 exec_time=206.01968455314636
done: p=4 run=1 p_test=10 exec_time=466.2335915565491
done: p=4 run=2 p_test=2 exec_time=5.0397419929504395
done: p=4 run=2 p_test=4 exec_time=22.5273277759552
done: p=4 run=2 p_test=6 exec_time=88.88311791419983
done: p=4 run=2 p_test=8 exec_time=217.62510752677917
done: p=4 run=2 p_test=10 exec_time=375.1414761543274
done: p=4 run=3 p_test=2 exec_time=4.475729465484619
done: p=4 run=3 p_test=4 exec_time=22.636247634887695
done: p=4 run=3 p_test=6 exec_time=75.69964838027954
done: p=4 run=3 p_test=8 exec_time=216.31004762649536
done: p=4 run=3 p_test=10 exec_time=403.13480281829834
done: p=4 run=4 p_test=2 exec_time=4.38765287399292
done: p=4 run=4 p_test=4 exec_time=23.045542001724243
done: p=4 run=4 p_test=6 exec_time=79.35905241966248
done: p=4 run=4 p_test=8 exec_time=24

In [ ]:
f1_scores

In [ ]:
plt.figure(figsize=(8,8))
plt.title('f1 scores for differen p')
plt.xlabel('assumed p')
plt.ylabel('mean f1')
handles = []
labels = []
for p in max_ps:
    plt.errorbar(x=f1_scores.loc[f1_scores['true_p']==p, 'p'],
                 y=f1_scores.loc[f1_scores['true_p']==p, 'mean_f1'],
                 yerr=f1_scores.loc[f1_scores['true_p']==p, 'std_f1'],
                 label='true p='+str(p))
plt.legend()
plt.show()